In [ ]:
# run after fresh restart
!pip -q install --upgrade --no-cache-dir \
  torch==2.5.1+cu121 torchvision==0.20.1+cu121 torchaudio==2.5.1 \
  --index-url https://download.pytorch.org/whl/cu121
!pip -q install --no-cache-dir \
  transformers==4.45.2 peft==0.12.0 accelerate==0.34.2 \
  bitsandbytes==0.43.3 triton sentencepiece "protobuf<5" datasets==2.21.0 \
  evaluate scikit-learn

import torch, platform, subprocess
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available(), "| Python:", platform.python_version())
try:
    print(subprocess.check_output(["nvidia-smi"], text=True))
except Exception as e:
    print("No nvidia-smi:", e)


In [ ]:
from google.colab import files
uploaded = files.upload()

import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
np.random.seed(42)

df = pd.read_csv(list(uploaded.keys())[0])
df = df[['title','description','requirements','fraudulent']].dropna()
df['text'] = df['title'].astype(str) + "\n" + df['description'].astype(str) + "\n" + df['requirements'].astype(str)
df['label_text'] = df['fraudulent'].map({0:"Real", 1:"Fake"})

train_df, test_df = train_test_split(
    df[['text','label_text']], test_size=0.2, random_state=42, stratify=df['label_text']
)

# balance
max_n = train_df['label_text'].value_counts().max()
train_df = (train_df
            .groupby('label_text', group_keys=False)
            .apply(lambda x: x.sample(max_n, replace=True, random_state=42))
            .sample(frac=1.0, random_state=42))  # shuffle

# keep TEST untouched
test_df  = test_df.sample(min(len(test_df), 400), random_state=42)

# if you want to hard-cap training for speed:
train_df = train_df.sample(min(len(train_df), 2000), random_state=42)

train_df.to_json("train.jsonl", orient="records", lines=True)
test_df.to_json("test.jsonl",  orient="records", lines=True)

print(train_df['label_text'].value_counts(), "\n---\n", test_df['label_text'].value_counts())


In [ ]:
from datasets import load_dataset
ds = load_dataset("json", data_files={"train":"train.jsonl","test":"test.jsonl"})

def format_example(ex):
    # keep labels as words
    return {"text": f"Decide if this job posting is Real or Fake:\n\n{ex['text']}\n\nAnswer: {ex['label_text']}"}

ds = ds.map(format_example, remove_columns=ds["train"].column_names)


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

base_model = "mistralai/Mistral-7B-Instruct-v0.2"

tok = AutoTokenizer.from_pretrained(base_model, use_fast=True)
if tok.pad_token is None: tok.pad_token = tok.eos_token
tok.padding_side = "right"

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# single load
max_memory = {0: "13GiB", "cpu": "24GiB"}
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb,
    device_map="auto",
    max_memory=max_memory,
    torch_dtype=torch.float16,
    attn_implementation="sdpa",
    low_cpu_mem_usage=True,
)
model.config.use_cache = False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model = prepare_model_for_kbit_training(model)

lora = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    bias="none", task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora)
model.print_trainable_parameters()

# 128 usually enough
MAXLEN = 128
def tokenize(batch):
    return tok(batch["text"], truncation=True, max_length=MAXLEN, padding="max_length")

tok_ds = ds.map(tokenize, batched=True, remove_columns=["text"])

import torch
from dataclasses import dataclass
@dataclass
class Collator:
    pad_token_id: int
    def __call__(self, feats):
        input_ids = torch.tensor([f["input_ids"] for f in feats], dtype=torch.long)
        attention_mask = torch.tensor([f["attention_mask"] for f in feats], dtype=torch.long)
        labels = input_ids.clone()
        labels[attention_mask == 0] = -100
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}
collator = Collator(pad_token_id=tok.pad_token_id)

args = TrainingArguments(
    output_dir="./mistral-qlora-fakejobs",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,   # ← effectively batch=4
    max_steps=160,                   # you can raise to 400 if need
    learning_rate=2e-4,
    warmup_steps=20,
    fp16=True,
    logging_steps=20,
    save_strategy="no",
    report_to=[],
    optim="paged_adamw_8bit",
    gradient_checkpointing=True,
    eval_strategy="no",              # ← avoids mid-train eval memory spikes
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tok_ds["train"],
    data_collator=collator,
)
trainer.train()

model.save_pretrained("mistral_finetuned_lora_t4")
tok.save_pretrained("mistral_finetuned_lora_t4")


In [ ]:
# calibration (clamped bias + class counts) (BASELINE)
import json, numpy as np, torch, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

device = next(model.parameters()).device
model.eval()
MAXLEN = 128

def first_id_for(text):
    return tok(text, add_special_tokens=False).input_ids[0]

rid = first_id_for(" Real"); fid = first_id_for(" Fake"); uid = first_id_for(" Uncertain")
label_ids = torch.tensor([rid, fid, uid], device=device)

def score_texts(texts, batch=8):
    outs = []
    with torch.no_grad():
        for i in range(0, len(texts), batch):
            enc = tok(texts[i:i+batch], return_tensors="pt",
                      truncation=True, max_length=MAXLEN, padding=True).to(device)
            logits = model(**enc, use_cache=False).logits[:, -1, :]
            outs.append(logits[:, label_ids])
    return torch.cat(outs, dim=0)

def decide(logits, bias_fake=0.0, p_uncertain=0.50, margin=0.10):
    l = logits.clone().float()
    l[:, 1] += bias_fake
    probs = torch.softmax(l, dim=-1)
    top_p, top_i = probs.max(dim=-1)
    second_p = probs.topk(2, dim=-1).values[:, 1]
    out = []
    for i in range(l.size(0)):
        if (top_p[i].item() < p_uncertain) or ((top_p[i]-second_p[i]).item() < margin):
            out.append("Uncertain")
        else:
            out.append(["Real","Fake","Uncertain"][top_i[i].item()])
    return out

def metrics_for(preds, gold, tiebreak="Real"):
    p2 = [p if p in ("Real","Fake") else tiebreak for p in preds]
    acc = accuracy_score(gold, p2)
    f1w = f1_score(gold, p2, average="weighted")
    return acc, f1w

# build
prompts, gold = [], []
with open("test.jsonl") as f:
    for line in f:
        ex = json.loads(line)
        gold.append(ex["label_text"])
        prompts.append(f"Decide if this job posting is Real or Fake.\n\n{ex['text']}\n\nAnswer:")

prom_calib, prom_eval, y_calib, y_eval = train_test_split(
    prompts, gold, test_size=0.8, random_state=42, stratify=gold
)
logits_calib = score_texts(prom_calib)
logits_eval  = score_texts(prom_eval)

# search moderate bias
best = (-1.0, 0.0)
for b in np.linspace(-0.8, +0.8, 33):  # narrower range avoids overcorrection
    preds_c = decide(logits_calib, bias_fake=b, p_uncertain=0.50, margin=0.10)
    p2 = [p if p in ("Real","Fake") else "Real" for p in preds_c]
    from sklearn.metrics import f1_score
    f1f = f1_score(y_calib, p2, pos_label="Fake")
    if f1f > best[0]:
        best = (f1f, b)

best_bias = float(best[1])
print(f"Chosen Fake-bias: {best_bias:+.2f}")

preds = decide(logits_eval, bias_fake=best_bias, p_uncertain=0.50, margin=0.10)
acc, f1w = metrics_for(preds, y_eval)
print(f"CALIBRATED — Acc (forced 2-label): {acc:.3f}  F1(weighted): {f1w:.3f}")

labels2 = ["Real","Fake"]
preds2 = [p if p in labels2 else "Real" for p in preds]
cm = pd.DataFrame(confusion_matrix(y_eval, preds2, labels=labels2),
                  index=labels2, columns=[f"Pred {l}" for l in labels2])
print(cm)

# show class counts
from collections import Counter
print("Pred counts:", Counter(preds))
u_rate = sum(1 for p in preds if p=="Uncertain") / len(preds)
print(f"Uncertain rate: {u_rate:.1%}")


In [ ]:
import gc, torch, psutil, os
gc.collect()
torch.cuda.empty_cache()
print("RAM used:", psutil.virtual_memory().percent, "%")


In [ ]:
# calibration (logistic calibrator + Uncertain by confidence) (TUNED)
import json, numpy as np, torch, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

device = next(model.parameters()).device
model.eval()
MAXLEN = 128  # keep in sync with training

def token_set_for(label_word: str):
    ids = set()
    for variant in [label_word, f" {label_word}"]:
        toks = tok(variant, add_special_tokens=False).input_ids
        if toks: ids.add(toks[0])
    return sorted(ids)

real_ids = token_set_for("Real")
fake_ids = token_set_for("Fake")
unc_ids  = token_set_for("Uncertain")

def slice_label_logits(next_logits):  # next_logits: [B,V]
    def max_col(ids):
        cols = [next_logits[:, i] for i in ids if i < next_logits.size(1)]
        if not cols:
            return torch.full((next_logits.size(0),), -1e9, device=next_logits.device)
        return torch.stack(cols, dim=1).max(dim=1).values
    return torch.stack([max_col(real_ids), max_col(fake_ids), max_col(unc_ids)], dim=1)  # [B,3]

def score_texts(texts, batch=8):
    outs = []
    with torch.no_grad():
        for i in range(0, len(texts), batch):
            enc = tok(texts[i:i+batch], return_tensors="pt",
                      truncation=True, max_length=MAXLEN, padding=True).to(device)
            next_logits = model(**enc, use_cache=False).logits[:, -1, :]
            outs.append(slice_label_logits(next_logits))
    return torch.cat(outs, dim=0)  # [N,3]

# build prompts
prompts, gold = [], []
with open("test.jsonl") as f:
    for line in f:
        ex = json.loads(line)
        gold.append(ex["label_text"])
        prompts.append(f"Decide if this job posting is Real or Fake.\n\n{ex['text']}\n\nAnswer:")

# 30% calibration / 70% evaluation
prom_calib, prom_eval, y_calib, y_eval = train_test_split(
    prompts, gold, test_size=0.7, random_state=42, stratify=gold
)

logits_calib = score_texts(prom_calib)  # [C,3]
logits_eval  = score_texts(prom_eval)   # [E,3]

# features for a tiny meta-classifier (no GPU strain)
def feats_from_logits(L):  # L: [N,3] -> features [lR, lF, lU, gap(F-R)]
    lR, lF, lU = L[:,0].cpu().numpy(), L[:,1].cpu().numpy(), L[:,2].cpu().numpy()
    gap = (L[:,1] - L[:,0]).cpu().numpy()
    return np.stack([lR, lF, lU, gap], axis=1)

Xc = feats_from_logits(logits_calib)
Xe = feats_from_logits(logits_eval)
yc = np.array([1 if y=="Fake" else 0 for y in y_calib])  # 1=Fake, 0=Real

clf = LogisticRegression(solver="liblinear", class_weight="balanced", max_iter=1000)
clf.fit(Xc, yc)

# predict fake
fake_prob = clf.predict_proba(Xe)[:,1]

# choose thresholds through a sweep to maximize weighted F1 on calibration
best = {"f1w": -1, "tau": 0.50, "conf_t": 0.55}
from sklearn.metrics import f1_score
for tau in np.linspace(0.40, 0.60, 21):
    for conf_t in np.linspace(0.50, 0.70, 21):
        p_cal = clf.predict_proba(Xc)[:,1]
        preds_tmp = []
        for p in p_cal:
            if max(p, 1-p) < conf_t:
                preds_tmp.append("Uncertain")
            else:
                preds_tmp.append("Fake" if p >= tau else "Real")
        forced = [("Fake" if p=="Fake" else "Real") for p in preds_tmp]
        f1w = f1_score(y_calib, forced, average="weighted", pos_label="Fake")
        if f1w > best["f1w"]:
            best = {"f1w": f1w, "tau": float(tau), "conf_t": float(conf_t)}

tau, conf_t = best["tau"], best["conf_t"]
print(f"Chosen thresholds → tau={tau:.2f} (Fake cut), conf_t={conf_t:.2f} (Uncertain)")

# apply on eval
preds = []
for p in fake_prob:
    if max(p, 1-p) < conf_t:
        preds.append("Uncertain")
    else:
        preds.append("Fake" if p >= tau else "Real")

def metrics_for(preds, gold, tiebreak="Real"):
    p2 = [p if p in ("Real","Fake") else tiebreak for p in preds]
    acc = accuracy_score(gold, p2)
    f1w = f1_score(gold, p2, average="weighted")
    return acc, f1w

acc, f1w = metrics_for(preds, y_eval)
print(f"CALIBRATED — Acc (forced 2-label): {acc:.3f}  F1(weighted): {f1w:.3f}")

labels2 = ["Real","Fake"]
p2 = [p if p in labels2 else "Real" for p in preds]
cm = pd.DataFrame(confusion_matrix(y_eval, p2, labels=labels2),
                  index=labels2, columns=[f"Pred {l}" for l in labels2])
print(cm)

from collections import Counter
print("Pred counts:", Counter(preds))
u_rate = sum(1 for p in preds if p=="Uncertain") / len(preds)
print(f"Uncertain rate: {u_rate:.1%}")


In [ ]:
# reasons for fakes
import re, json, html
from urllib.parse import urlparse

def norm_text(t: str) -> str:
    if not isinstance(t, str):
        t = str(t)
    t = html.unescape(t)
    t = re.sub(r"<[^>]+>", " ", t)           # strip HTML tags
    t = re.sub(r"\s+", " ", t).strip()
    # common obfuscations
    t = re.sub(r"\b(?:\(|\[)?at(?:\)|\])?\b", "@", t, flags=re.I)
    t = re.sub(r"\b(?:\(|\[)?dot(?:\)|\])?\b", ".", t, flags=re.I)
    t = re.sub(r"g\s*m\s*a\s*i\s*l", "gmail", t, flags=re.I)
    t = re.sub(r"y\s*a\s*h\s*o\s*o", "yahoo", t, flags=re.I)
    t = re.sub(r"o\s*u\s*t\s*l\s*o\s*o\s*k", "outlook", t, flags=re.I)
    return t

# regexes & helpers
EMAIL    = re.compile(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.(?:com|net|org|io|co|edu)\b", re.I)
FREE_DOM = re.compile(r"@(gmail|yahoo|outlook|hotmail|protonmail)\.", re.I)
PHONE    = re.compile(r"\b(?:\+?\d{1,3}[-.\s]?)?(?:\(?\d{3}\)?[-.\s]?)?\d{3}[-.\s]?\d{4}\b")
URL      = re.compile(r"https?://\S+")
SHORTEN  = re.compile(r"(bit\.ly|tinyurl\.com|linktr\.ee|t\.co|is\.gd|goo\.gl|forms\.gle)", re.I)

CHAT_APPS = re.compile(r"\b(telegram|whatsapp|signal|skype|wechat|discord)\b", re.I)
TEXT_ONLY = re.compile(r"\b(interview|onboarding|orientation)\b.*\b(chat|text|telegram|whatsapp|signal|discord)\b", re.I)

PAY_APPS  = re.compile(r"\b(zelle|cash\s*app|cashapp|venmo|apple\s*pay|google\s*pay|moneygram|western\s*union)\b", re.I)
CRYPTO    = re.compile(r"\b(crypto|bitcoin|btc|usdt|wallet|metamask)\b", re.I)
FEE_LANG  = re.compile(r"\b(upfront|processing|training|registration|equipment)\s+fee\b", re.I)
DEPOSIT   = re.compile(r"\b(cash(?:\s|-)?ier'?s?\s*check|mobile\s*deposit|check\s*deposit)\b", re.I)
RESHIP    = re.compile(r"\b(reship|package\s*handler|parcel\s*forward|warehouse\s*at\s*home|logistics\s*agent)\b", re.I)

GOOGLE_FORMS = re.compile(r"\b(forms\.gle|docs\.google\.com/forms)\b", re.I)
GOOGLE_DRIVE = re.compile(r"\b(drive\.google\.com)\b", re.I)
CLICKY       = re.compile(r"\b(click\s+(?:the\s+)?link|apply\s+via\s+link|fill\s+the\s+form|complete\s+the\s+form)\b", re.I)

TOO_GOOD  = re.compile(r"\b(no\s*experience|immediate|urgent|quick\s*money|daily\s*pay|weekly\s*pay|work\s*from\s*home|remote\s*only)\b", re.I)
NO_INTERV = re.compile(r"\b(no\s*interview|required\s*immediately)\b", re.I)
VISA      = re.compile(r"\bvisa\s*sponsor(ship)?\b", re.I)

PAY_HIGH  = re.compile(r"\$\s*([5-9]\d|[1-9]\d{2,})\s*/\s*(hr|hour|week|day)", re.I)

# common scam role names
ROLE_BAIT = re.compile(r"\b(data\s*entry|typist|repack|shipping\s*clerk|personal\s*assistant|virtual\s*assistant|payroll\s*assistant)\b", re.I)

def basic_features(t: str):
    """Return small feature dict for concrete cues."""
    n_chars = len(t)
    words   = re.findall(r"\b\w+\b", t)
    n_words = len(words)
    caps_tokens = [w for w in words if sum(c.isupper() for c in w) >= 3 and sum(c.isalpha() for c in w) >= 3]
    caps_ratio  = len(caps_tokens) / max(1, n_words)

    emails = EMAIL.findall(t)
    free_emails = [e for e in emails if FREE_DOM.search(e)]
    urls = URL.findall(t)
    short_urls = [u for u in urls if SHORTEN.search(u)]
    phones = PHONE.findall(t)

    return {
        "n_chars": n_chars, "n_words": n_words,
        "caps_ratio": caps_ratio,
        "n_emails": len(emails), "n_free_emails": len(free_emails),
        "n_urls": len(urls), "n_short": len(short_urls),
        "n_phones": len(phones),
    }

def feature_reasons(t: str, max_cues=3):
    cues = []
    T = norm_text(t)
    Tl = T.lower()

    # pattern hits
    if TEXT_ONLY.search(T):     cues.append("interview/onboarding over text/chat app")
    if CHAT_APPS.search(T):     cues.append("redirects to chat app for hiring")
    if FEE_LANG.search(T):      cues.append("mentions upfront/processing/training/equipment fee")
    if PAY_APPS.search(T):      cues.append("requests payment via Zelle/CashApp/Venmo/etc.")
    if CRYPTO.search(T):        cues.append("mentions crypto payments/wallet")
    if DEPOSIT.search(T):       cues.append("check-deposit / cashier’s check scheme")
    if RESHIP.search(T):        cues.append("reshipping / at-home logistics")
    if GOOGLE_FORMS.search(T):  cues.append("collects info via Google Forms")
    if GOOGLE_DRIVE.search(T):  cues.append("shares Google Drive link for onboarding")
    if CLICKY.search(T):        cues.append("pressures to click external link/form")
    if NO_INTERV.search(T):     cues.append("claims no interview / immediate start")
    if TOO_GOOD.search(T):      cues.append("too-good-to-be-true pitch (quick money/no experience/remote-only)")
    if VISA.search(T):          cues.append("vague visa sponsorship claim")
    if ROLE_BAIT.search(T):     cues.append("role matches common scam bait (data entry/personal assistant/etc.)")
    if PAY_HIGH.search(T):      cues.append("unusually high pay claim")

    # feeature counts
    feats = basic_features(T)
    if feats["n_free_emails"] > 0:
        cues.append("uses free email domain for hiring")
    elif feats["n_emails"] > 0:
        # any email but not company domain is still suspicious if no company is named
        cues.append("direct email contact instead of company domain portal")

    if feats["n_phones"] > 0:
        cues.append("phone/text contact provided")

    if feats["n_urls"] >= 3:
        cues.append(f"contains {feats['n_urls']} links (possible off-platform funnel)")
    elif feats["n_urls"] >= 1 and feats["n_short"] >= 1:
        cues.append("uses URL shortener (bit.ly/tinyurl/etc.)")

    if feats["caps_ratio"] > 0.10:
        cues.append("excessive ALL-CAPS wording")
    if feats["n_words"] < 60:
        cues.append("very short description, lacks detail")
    elif feats["n_words"] > 800:
        cues.append("unusually long description (padding/boilerplate)")

    # remove duplicates
    out = []
    seen = set()
    for c in cues:
        if c and c not in seen:
            out.append(c)
            seen.add(c)
        if len(out) >= max_cues:
            break

    if not out:
        out = ["signals unclear"]
    return out

# reasons for fake
raw_eval_texts = [p.split("Real or Fake.\n\n",1)[1].rsplit("\n\nAnswer:",1)[0] for p in prom_eval]

fake_rows = []
for i, label in enumerate(preds):
    if label == "Fake":
        reasons = feature_reasons(raw_eval_texts[i], max_cues=3)
        fake_rows.append({"idx": i, "reasons": reasons})

print(f"Explained {len(fake_rows)} Fake items. Sample:")
print(fake_rows[:10])


In [ ]:
# Clean a notebook so GitHub can render it (removes metadata.widgets + clears outputs)
import os, re, html, json, glob, time
import nbformat as nbf

def pick_notebook():
    # 1) try COLAB_NOTEBOOK_PATH
    nb_path = os.environ.get("COLAB_NOTEBOOK_PATH")
    if nb_path and os.path.isfile(nb_path):
        return nb_path
    # 2) try newest .ipynb in /content
    cands = sorted(glob.glob("/content/*.ipynb"), key=lambda p: os.path.getmtime(p), reverse=True)
    if cands:
        print("Auto-selected most recent .ipynb:", os.path.basename(cands[0]))
        return cands[0]
    # 3) ask you to upload a notebook
    try:
        from google.colab import files
        print("No .ipynb found in /content — please select your notebook file to clean…")
        uploaded = files.upload()  # opens picker
        if uploaded:
            fname = list(uploaded.keys())[0]
            return "/content/" + fname
    except Exception as e:
        pass
    raise SystemExit("No notebook file found. Upload a .ipynb and re-run this cell.")

nb_path = pick_notebook()
print("Cleaning:", nb_path)

nb = nbf.read(nb_path, as_version=4)

# Remove notebook-level widgets metadata
nb.metadata.pop("widgets", None)

# Clean per-cell outputs + any cell-level widget metadata
for cell in nb.cells:
    if hasattr(cell, "metadata"):
        cell.metadata.pop("widgets", None)
    if "outputs" in cell:
        cell["outputs"] = []
    if "execution_count" in cell:
        cell["execution_count"] = None

# Write cleaned copy
root, ext = os.path.splitext(nb_path)
clean_path = root + "_CLEAN.ipynb"
nbf.write(nb, clean_path)
print("Wrote:", clean_path)

# (Optional) also export HTML for easy viewing
try:
    import subprocess, sys
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "nbconvert==7.16.6"], check=True)
    subprocess.run(["jupyter", "nbconvert", "--to", "html", "--no-input", "--no-prompt", clean_path], check=True)
    print("HTML written:", clean_path.replace(".ipynb", ".html"))
except Exception as e:
    print("HTML export skipped:", e)

print("\nNext steps:")
print("1) In the left Files pane, right-click the *_CLEAN.ipynb and Download it.")
print("2) On GitHub → your repo → Add file → Upload files → drop the *_CLEAN.ipynb (and optional .html).")
